In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import os

os.chdir("..")

import funciones.parser  as ps

In [2]:
df = ps.branch_data_df('data/prueba_sin_sobrecargas.raw')
df

,I,J,R,X,B
0,'ACO150_B 150.00','PBE150BX 150.00',0.002933,0.009957,0.00376
1,'ACO150_B 150.00','PBE150RX 150.00',0.002933,0.009957,0.00376
2,'ARA150_B 150.00','CLFAUX150_B 150.00',0.015066,0.045404,0.01581
3,'ARA150_B 150.00','TGO150_B 150.00',0.033038,0.099566,0.03467
4,'ARB150P1 150.00','ARB150_B 150.00',0.000920,0.003720,0.00123
...,...,...,...,...,...
138,'SCA150_B 150.00','SCAROC_X 150.00',0.003609,0.013645,0.00517
139,'SGDE.ARG 500.00','SU5500_B 500.00',0.000040,0.000410,0.03890
140,'SJ5500_B 500.00','SU5500_B 500.00',0.001710,0.016060,1.52925
141,'TAB150_B 150.00','TAC150_B 150.00',0.007609,0.022935,0.00797


In [3]:
def get_Admittances(branch_df):
  Admitance = np.empty((branch_df.shape[0],2),dtype = 'complex_')
  Admitance[:,0] = 1/(branch_df['R']+branch_df['X']*1j)
  Admitance[:,1] = branch_df['B']/2*1j
  Admitance = pd.DataFrame(Admitance,columns=['y','ysh'])

  Admitance['I'] = branch_df['I']
  Admitance['J'] = branch_df['J']
  Admitance = Admitance[['I','J','y','ysh']]
  return Admitance

def get_graph(edges_df):
  new_rows = []
  for _, row in edges_df.iterrows():
      source, sink, _, self_loop_weight = row
      new_rows.append({'source': source, 'target': source, 'weight': self_loop_weight})
      new_rows.append({'source': sink, 'target': sink, 'weight': self_loop_weight})
  new_edges_df = edges_df.drop(columns=['ysh'])
  new_edges_df.rename(columns={'I': 'source', 'J': 'target','y': 'weight'}, inplace=True)
  new_rows = pd.DataFrame(new_rows)
  new_edges_df = pd.concat([new_edges_df,new_rows], axis=0)
  new_edges_df_2 = new_edges_df.groupby(['source', 'target'])['weight'].sum().reset_index()
  G = nx.from_pandas_edgelist(new_edges_df_2, edge_attr=True)
  return G

def get_Ybus(G):
  Y = nx.laplacian_matrix(G).toarray()
  for i, node in enumerate(G.nodes()):
      degree = G.degree(node,weight='weight')
      Y[i, i] += (degree - Y[i, i])  / 2
  return Y

In [5]:
Admitance = get_Admittances(df)
G = get_graph(Admitance)
Ybus = get_Ybus(G)

# Ybus
# for edge in G.edges():
#     weight_complex = G.edges[edge]['weight']
#     weight_magnitude = abs(weight_complex)
#     G.edges[edge]['weight'] = weight_magnitude

# # Definir el layout y visualizar los nodos
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_size=1000, node_color='skyblue', font_size=10, font_color='darkblue', font_weight='bold')

# # Dibujar los enlaces con colores y grosores basados en los valores transformados
# nx.draw_networkx_edges(G, pos, edge_color='gray')

# # Agregar anotaciones o leyendas si es necesario

# # Mostrar la visualización
# plt.show()


In [ ]:
nombres_ordenados = list(G.nodes)
nombres_ordenados = [valor.replace('150.00', '') for valor in nombres_ordenados]
nombres_ordenados = [valor.replace('500.00', '') for valor in nombres_ordenados]
nombres_ordenados = [valor.replace('\'', '') for valor in nombres_ordenados]
nombres_ordenados = [valor.replace(' ', '') for valor in nombres_ordenados]
nombres_ordenados

In [ ]:
datos = '/content/drive/MyDrive/Proyecto/Datos/prueba_sin_sobrecargas.raw'
bus_data = ps.bus_data_df(datos)[['NAME', 'BASKV', 'VM', 'VA']]
bus_data = bus_data.set_index('NAME')
bus_data.index = bus_data.index.str.replace('\'', '', regex=False)
bus_data.index = bus_data.index.str.replace(' ', '', regex=False)
bus_data['V (KV)'] = bus_data['BASKV'] * bus_data['VM'] * (np.cos(np.deg2rad(bus_data['VA'])) + np.sin(np.deg2rad(bus_data['VA'])) * 1j)
bus_data = bus_data.reindex(nombres_ordenados)
V = bus_data['V (KV)'].to_numpy()
diagV = np.diag(V)